
# **Introducción a Python para Análisis de Datos**
## Capítulo 3: Manipulación de Datos
---
**Autor:** Juan Martin Bellido  

**Descripción**  
En este capítulo se presentan operaciones típicas de manipulación de datos. Se asume conocimientos básicos previos en tratamientos de DataFrames.

**¿Feedback? ¿comentarios?** Por favor compártelo conmigo escribiéndome por [LinkedIn](https://www.linkedin.com/in/jmartinbellido/)  

**Material Adicional**

* [Guía Python Importar/Exportar DataFrames](https://datawizards.es/contenido/codigo-para-analisis-de-datos/guias/python-importar-exportar)
* [Comandos Jupyter Notebook](https://datawizards.es/contenido/codigo-para-analisis-de-datos/guias/comandos-rapidos-jupyter)
* [Sintaxis Markdown](https://datawizards.es/contenido/codigo-para-analisis-de-datos/guias/sintaxis-markdown)



## INDICE
---
1. Agregación de datos
2. Trabajar con datos históricos
3. Unir tablas de datos
4. Pivotar tablas de datos
5. Ejercicios


Convenciones utilizadas en este documento
> 👉 *Esto es una nota u observación*

> ⚠️ *Esto es una advertencia*

# 1. Agregación de datos
---


En el contexto del análisis de datos, *agregar* data significa realizar cálculos que permitan resumir información, abstrayendo conocimiento de data que se encuentra en estado bruto. 

> 👉 Se trata del ejercicio que hacemos cada vez que utilizamos una tabla dinámica (pivot table) en Excel


Al realizar una agregación debemos establecer *el tipo de cálculo que buscamos realizar*. Para ello, utilizamos funciones específicas: *funciones de agregación*. 

*Funciones de agregación*

| Función     | Descripción                     |
|-------------|---------------------------------|
| count       | Contar filas no nulas           |
| nunique     | Contar valores únicos           |
| sum         | Suma de valores                 |
| mean        | Media aritmética                |
| median      | Mediana                         |
| mode        | Moda                            |
| min         | Valor mínimo                    |
| max         | Valor máximo                    |

In [ ]:
import pandas as pd

In [ ]:
# importamos df
df_jamesbond = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/jamesbond.csv")
df_jamesbond.dtypes

Year                   int64
Actor                 object
Director              object
Box Office           float64
Budget               float64
Bond Actor Salary    float64
dtype: object

### Agregaciones básicas

Comenzaremos realizando agregaciones simplemente utilizando funciones de aggregación sobre dataframes.

```
df.agg_func()
```


In [ ]:
# aplicamos una función de agregación sobre el dataframe
df_jamesbond.max()
# obtenemos los valores max para cada una de las variables

Year                           2015
Actor                Timothy Dalton
Director              Terence Young
Box Office                    943.5
Budget                        206.3
Bond Actor Salary              17.9
dtype: object

In [ ]:
# seleccionamos una variable y aplicamos función de agregación
df_jamesbond.Budget.sum().round()
# anidamos un .round() para redondear decimales

2099.0

In [ ]:
# seleccionamos una variable y aplicamos función de agregación
df_jamesbond.Actor.nunique()
# existen 7 actores de James Bond (únicos) en el dataframe

7

### El método .agg()

Pandas incluye un método dedicado a agregar data.

```
df.agg({'column':'agg_func'})
```
> 👉 Utilizamos un *dictionary* para establacer sobre qué variable realizar qué operación

In [ ]:
# agregamos data utilizando .agg()
df_jamesbond.agg({'Actor':"nunique"})

Actor    7
dtype: int64

In [ ]:
# agregamos la columna "box office", aplicando funciones min(), max() y mean()
df_jamesbond.agg({'Box Office':["min","max","mean"]})
# en este caso, hemos utilizado más de una función de agregación sobre la misma variable

,Box Office
min,250.900000
max,943.500000
mean,491.611538


In [ ]:
# agregamos más de una columna añadiendo múltiples elementos en nuestro dictionary
df_jamesbond.agg({
    'Budget':["min","max","median"],           
    'Bond Actor Salary':["min","max","median"]   
    })

,Budget,Bond Actor Salary
min,7.00,0.6
max,206.30,17.9
median,60.05,5.5


### Agregaciones agrupadas

Al realizar agregaciones, frecuentemente agrupamos observaciones en una o más variables categóricas. 

El método `.groupby()` permite agrupar observaciones para posteriormente agregar según grupos.

```
object.groupby("column")
```

In [ ]:
# agregamos dos métricas, según una variable agrupada
df_jamesbond.groupby("Actor")[["Bond Actor Salary","Box Office"]].max()

,Bond Actor Salary,Box Office
Actor,,
Daniel Craig,14.5,943.5
David Niven,NaN,315.0
George Lazenby,0.6,291.5
Pierce Brosnan,17.9,518.5
Roger Moore,9.1,535.0
Sean Connery,5.8,848.1
Timothy Dalton,7.9,313.5


In [ ]:
# repetimos ejercicio utilizando la función .agg()
df_jamesbond.groupby("Actor").agg(
    {"Bond Actor Salary":"max","Box Office":"max"}
)

,Bond Actor Salary,Box Office
Actor,,
Daniel Craig,14.5,943.5
David Niven,NaN,315.0
George Lazenby,0.6,291.5
Pierce Brosnan,17.9,518.5
Roger Moore,9.1,535.0
Sean Connery,5.8,848.1
Timothy Dalton,7.9,313.5


In [ ]:
# agregamos salario máximo y media de box office, según actor
## cambiamos los nombres de las variables agregadas
df = df_jamesbond.groupby("Actor").agg(
    {"Bond Actor Salary":"max","Box Office":"mean"}
).sort_values("Bond Actor Salary", ascending=False)

## muchas veces, los nombres que obtenemos por defecto no son representativos de la información que almacenan
df = df.rename(columns={'Bond Actor Salary':'total_bond_salary','Box Office':'total_box_office'})

df # invocamos el objeto

,total_bond_salary,total_box_office
Actor,,
Pierce Brosnan,17.9,471.650000
Daniel Craig,14.5,691.475000
Roger Moore,9.1,422.957143
Timothy Dalton,7.9,282.200000
Sean Connery,5.8,571.114286
George Lazenby,0.6,291.500000
David Niven,NaN,315.000000


In [ ]:
# agregamos múltiples métricas utilizando distintas funciones
## al momento de ordenar, debemos utilizar un tuple para poder especificar (i) variable y (ii) función
df_jamesbond.groupby("Actor").agg(
    {"Bond Actor Salary":["max","sum","mean","size"]
    ,"Budget":["max","min"]}
).sort_values(("Bond Actor Salary","max"),ascending=False) # al ordenar el df, debemos utilizar un "tuple" para definir criterio


Bond Actor Salary                       Budget       
                             max   sum       mean size    max    min
Actor                                                               
Pierce Brosnan              17.9  46.5  11.625000    4  158.3   76.9
Daniel Craig                14.5  25.9   8.633333    4  206.3  145.3
Roger Moore                  9.1  16.9   8.450000    7   91.5   27.7
Timothy Dalton               7.9  13.1   6.550000    2   68.8   56.7
Sean Connery                 5.8  20.3   3.383333    7   86.0    7.0
George Lazenby               0.6   0.6   0.600000    1   37.3   37.3
David Niven                  NaN   0.0        NaN    1   85.0   85.0

In [ ]:
# podemos utilizar más de una variable al agregar
## en tal caso, las observaciones se agruparán según estos dos campo combinados
df_jamesbond.groupby(["Director","Actor"]).agg(
    {"Box Office":"median"}
)

Box Office
Director           Actor                     
Guy Hamilton       Roger Moore         397.15
                   Sean Connery        631.45
Irvin Kershner     Sean Connery        380.00
John Glen          Roger Moore         373.80
                   Timothy Dalton      282.20
Ken Hughes         David Niven         315.00
Lee Tamahori       Pierce Brosnan      465.40
Lewis Gilbert      Roger Moore         534.00
                   Sean Connery        514.20
Marc Forster       Daniel Craig        514.20
Martin Campbell    Daniel Craig        581.50
                   Pierce Brosnan      518.50
Michael Apted      Pierce Brosnan      439.50
Peter R. Hunt      George Lazenby      291.50
Roger Spottiswoode Pierce Brosnan      463.20
Sam Mendes         Daniel Craig        835.10
Terence Young      Sean Connery        543.80

# 2. Trabajar con datos históricos
---
Python no reconoce las fechas como tipo de variable de forma nativa. Para poder operar con ellos, debemos importar librerías específicas.

In [ ]:
import pandas as pd

### Operando con fechas en un DataFrame


In [ ]:
# importamos df
df_amzn = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/amzn_stocks.csv")
df_amzn.dtypes

Date       object
Open      float64
High      float64
Low       float64
Close     float64
Volume      int64
dtype: object

In [ ]:
# usamos la función pd.to_datetime() para convertir una columna en tipo objeto
df_amzn['Date'] = pd.to_datetime(df_amzn['Date'])
df_amzn.dtypes

Date      datetime64[ns]
Open             float64
High             float64
Low              float64
Close            float64
Volume             int64
dtype: object

In [ ]:
# en caso de que Pandas no identifique automáticamente el formato de fecha, podemos especificarlo editando un parámetro opcional
df_amzn['Date'] = pd.to_datetime(df_amzn['Date'],format='%Y-%m-%d')
df_amzn.dtypes

Date      datetime64[ns]
Open             float64
High             float64
Low              float64
Close            float64
Volume             int64
dtype: object

In [ ]:
# creamos una nueva columna con el año
df_amzn['year'] = df_amzn['Date'].dt.year

In [ ]:
# creamos una nueva columna con el año
df_amzn['month'] = df_amzn['Date'].dt.month

In [ ]:
# creamos una nueva columna con el nombre de día
df_amzn['day_name'] = df_amzn['Date'].dt.day_name()

In [ ]:
# nueva columna con el mes (texto)
df_amzn['month_name'] = df_amzn['Date'].dt.month_name()

In [ ]:
# creamos una nueva columna con el número de semana
df_amzn['week_num'] = df_amzn['Date'].dt.isocalendar().week

In [ ]:
# visualizamos el objeto
df_amzn.head()

,Date,Open,High,Low,Close,Volume,year,month,day_name,month_name,week_num
0,2010-01-04,136.25,136.61,133.14,133.90,7600543,2010,1,Monday,January,1
1,2010-01-05,133.43,135.48,131.81,134.69,8856456,2010,1,Tuesday,January,1
2,2010-01-06,134.60,134.73,131.65,132.25,7180977,2010,1,Wednesday,January,1
3,2010-01-07,132.01,132.32,128.80,130.00,11030124,2010,1,Thursday,January,1
4,2010-01-08,130.56,133.68,129.03,133.52,9833829,2010,1,Friday,January,1


### Agregando data histórica
Podemos agregar data agrupando según componentes de fecha que hayamos calculado, utilizándolos como variables categóricas.


In [ ]:
# utilizamos los componentes "año" y "número de semana" para agrupar nuestra agregación
# calculamos la media semanal
df_amzn.groupby(['year','week_num']).agg({'Open':'mean'})

Open
year week_num           
2010 1          133.3700
     2          129.5660
     3          126.5475
     4          123.5780
     5          118.7220
...                  ...
2017 25        1003.6940
     26         987.3720
     27         967.1325
     28         997.1340
     29        1004.6900

[394 rows x 1 columns]

In [ ]:
# calculamos la media anual
output = df_amzn.groupby('year').agg({'Open':'mean'})
output

,Open
year,
2010,138.971036
2011,196.658413
2012,220.011040
2013,297.866270
2014,332.799841
2015,478.123532
2016,699.756587
2017,900.935259


El método `.shift()` permite tomar el valor de X filas anteriores.

In [ ]:
# utilizamos el método .shift() para crear una nueva columna que capte el valor del año anterior
output['open_1YA'] = output['Open'].shift(1)
output

,Open,open_1YA
year,,
2010,138.971036,NaN
2011,196.658413,138.971036
2012,220.011040,196.658413
2013,297.866270,220.011040
2014,332.799841,297.866270
2015,478.123532,332.799841
2016,699.756587,478.123532
2017,900.935259,699.756587


In [ ]:
# calculamos crecimiento anual (%)
output['yoy_growth'] = (output['Open'] / output['open_1YA']) - 1
output['yoy_growth'] = output['yoy_growth'].round(2)
output

,Open,open_1YA,yoy_growth
year,,,
2010,138.971036,NaN,NaN
2011,196.658413,138.971036,0.42
2012,220.011040,196.658413,0.12
2013,297.866270,220.011040,0.35
2014,332.799841,297.866270,0.12
2015,478.123532,332.799841,0.44
2016,699.756587,478.123532,0.46
2017,900.935259,699.756587,0.29


# 3. Unir tablas de datos
---
Con frecuencia nos vemos obligados a unir dos o más DataFrames. 

Podemos distinguir entre distintas operaciones:
* Unir tablas por filas; llamamos estas operaciones *unions*
* Unir por columnas, según algún campo en común; llamamos estas operaciones *joins*
* Simplemente unir por columnas, sin ningún campo en común



In [ ]:
# importamos df
df_jamesbond = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/jamesbond.csv")
df_jamesbond.dtypes

Film                  object
Year                   int64
Actor                 object
Director              object
Box Office           float64
Budget               float64
Bond Actor Salary    float64
dtype: object

### Unir tablas por filas

Uniremos dos dataframes utilizando la función `pd.concat()`. Nota: los datasets deben contener las mismas columnas.

Unions

In [ ]:
pd.concat([df_jamesbond.head(2),df_jamesbond.tail(2)])

,Film,Year,Actor,Director,Box Office,Budget,Bond Actor Salary
0,Dr. No,1962,Sean Connery,Terence Young,448.8,7.0,0.6
1,From Russia with Love,1963,Sean Connery,Terence Young,543.8,12.6,1.6
24,Skyfall,2012,Daniel Craig,Sam Mendes,943.5,170.2,14.5
25,Spectre,2015,Daniel Craig,Sam Mendes,726.7,206.3,NaN


### Unir tablas por columnas, según campos en común

La función `.merge()` permite instrumentar *joins*; es decir, unir dos DataFrames según columnas utilizando campo(s) en común. 

> 👉 El método `.merge()` permite instrumentar distintos tipos jode joins (left, right, inner, full)

```
df_1.merge(df_2,how='left')
```

> 👉 En caso de que los *keys* (columnas que usamos para unir) tengan nombres distintos en cada tabla, podemos editar los parámetros `left_on` y `right_on`

```
df_1.merge(df_2,how='inner',left_on='key',right_on='key')
```

In [ ]:
# importamos dataframe con data de albumes
df_albums = pd.read_csv('https://data-wizards.s3.amazonaws.com/datasets/rolling_stones_top_metal_albums.csv')
df_albums.dtypes

AlbumID_Rank              int64
Artist                   object
Album                    object
Release_Year              int64
Spotify_Album            object
Description              object
wiki                     object
Duration                 object
Minutes                   int64
Seconds                   int64
Total_Seconds             int64
Label                    object
Sub_Metal_Genre          object
Rating                  float64
Rolling_Stone_Rating     object
dtype: object

In [ ]:
# importamos dataframe con data de canciones
df_songs = pd.read_csv('https://data-wizards.s3.amazonaws.com/datasets/rolling_stones_top_metal_songs.csv')
df_songs.dtypes

Song Index       int64
Song            object
Artist          object
Track No.        int64
AlbumID_Rank     int64
BPM              int64
Energy           int64
Dance            int64
Loud             int64
Valence          int64
Duration        object
Acoustic         int64
Popularity       int64
Spotify_Song    object
dtype: object

In [ ]:
df_songs[['Song','Artist','AlbumID_Rank']].merge(
    df_albums[['AlbumID_Rank','Album','Release_Year']]  # df secundario
    ,how='left'                                         # definimos el criterio en el join, en este caso elegimos "left"
    ,left_on='AlbumID_Rank'                             # definimos la key (columna) en la tabla principal 
    ,right_on='AlbumID_Rank'                            # key en nuestra tabla secundaria
)

,Song,Artist,AlbumID_Rank,Album,Release_Year
0,War Pigs / Luke's Wall - 2014 Remaster,Black Sabbath,1,Paranoid,1970
1,Paranoid - 2016 Remaster,Black Sabbath,1,Paranoid,1970
2,Planet Caravan - 2013 Remaster,Black Sabbath,1,Paranoid,1970
3,Iron Man - 2014 Remaster,Black Sabbath,1,Paranoid,1970
4,Electric Funeral - 2013 Remaster,Black Sabbath,1,Paranoid,1970
...,...,...,...,...,...
1045,Sidewinder,Avenged Sevenfold,100,City of Evil,2005
1046,The Wicked End,Avenged Sevenfold,100,City of Evil,2005
1047,Strength of the World,Avenged Sevenfold,100,City of Evil,2005
1048,Betrayed,Avenged Sevenfold,100,City of Evil,2005


### Unir tablas por columnas, sin campos en común

In [ ]:
# importamos df
df_jamesbond = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/jamesbond.csv")
df_jamesbond.dtypes

Film                  object
Year                   int64
Actor                 object
Director              object
Box Office           float64
Budget               float64
Bond Actor Salary    float64
dtype: object

In [ ]:
df_1 = df_jamesbond[['Film','Actor','Director']]
df_1.head()

,Film,Actor,Director
0,Dr. No,Sean Connery,Terence Young
1,From Russia with Love,Sean Connery,Terence Young
2,Goldfinger,Sean Connery,Guy Hamilton
3,Thunderball,Sean Connery,Terence Young
4,Casino Royale,David Niven,Ken Hughes


In [ ]:
df_2 = df_jamesbond[['Box Office','Budget']]
df_2.head()

,Box Office,Budget
0,448.8,7.0
1,543.8,12.6
2,820.4,18.6
3,848.1,41.9
4,315.0,85.0


In [ ]:
df_3 = df_1.join(df_2)
df_3.head()

,Film,Actor,Director,Box Office,Budget
0,Dr. No,Sean Connery,Terence Young,448.8,7.0
1,From Russia with Love,Sean Connery,Terence Young,543.8,12.6
2,Goldfinger,Sean Connery,Guy Hamilton,820.4,18.6
3,Thunderball,Sean Connery,Terence Young,848.1,41.9
4,Casino Royale,David Niven,Ken Hughes,315.0,85.0


# 4. Pivotar tablas de datos
---
En ocasiones, nos encontamos con tablas de datos que estructuran la información de forma inadecuada, dificultando la exploración y manipulación. Debemos mantener la unidad de observación lo más simple posible, de forma tal de favorecer la interpretación de la tabla de datos.


### De formato *long* a *wide*

El método *pivot* nos permite separar una variable en múltiples columnas, convirtiendo una tabla *long* en *wide*.

```
df.pivot(
  index= [] # columnas que no deseamos modificar
  ,columns=[] # columnas que deseamos expandir
  ,values='col_name' # nombre métrica
)
```

In [ ]:
# importamos DataFrame
df_dummy_long_table = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/dummy_long_table.csv")
df_dummy_long_table

,country,year,type,count
0,Afghanistan,1999,cases,745
1,Afghanistan,1999,population,19987071
2,Afghanistan,2000,cases,2666
3,Afghanistan,2000,population,20595360
4,Brazil,1999,cases,37737
5,Brazil,1999,population,172006362
6,Brazil,2000,cases,80488
7,Brazil,2000,population,174504898
8,China,1999,cases,212258
9,China,1999,population,1272915272


In [ ]:
# utilizamos el método .pivot() para convertir tabla "long" en "wide"
df_dummy_long_table.pivot(index='country',columns=['year','type'],values='count')

year           1999                2000            
type          cases  population   cases  population
country                                            
Afghanistan     745    19987071    2666    20595360
Brazil        37737   172006362   80488   174504898
China        212258  1272915272  213766  1280428583

### De formato *wide* a *long*

El método *melt* nos permite agrupar múltiples variables, reduciendo el número de columnas y aumentando la cantidad de filas.

```
df.melt(
  id_vars = [] # columnas que no deseamos modificar
  ,value_vars = [] # columnas que deseamos agrupar
  ,var_name = 'col_name' # nombre variable categórica
  ,value_name = 'col_name' # nombre métrica
)
```



In [ ]:
# importamos tabla de datos
df_dummy_wide_table = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/dummy_wide_table.csv")
df_dummy_wide_table

,China,India,United States,Indonesia,Pakistan,Brazil,Nigeria,Bangladesh,Russia,Mexico
0,1439323776,1380004385,331002651,273523615,220892340,212559417,206139589,164689383,145934462,128932753


In [ ]:
# utilizamos el método .melt() para convertir df "wide" a "long"
df_dummy_wide_table.melt(var_name='country',value_name='population')

,country,population
0,China,1439323776
1,India,1380004385
2,United States,331002651
3,Indonesia,273523615
4,Pakistan,220892340
5,Brazil,212559417
6,Nigeria,206139589
7,Bangladesh,164689383
8,Russia,145934462
9,Mexico,128932753


# 5. Ejercicios
---
> 👉 Puedes encontrar las soluciones a los ejercicios [aquí](https://nbviewer.org/github/jmartinbellido/Python-Curso-Introductorio/blob/main/Capitulo%203%20Ejercicios.ipynb)

### Ejercicio #1

##### 1.A Calcular suma total de facturación (revenue), según sector
##### 1.B Repetir el ejercicio anterior, pero filtrando únicamente por sectores *Technology, Energy y Retailing*

> Dataset https://data-wizards.s3.amazonaws.com/datasets/fortune1000.csv


In [ ]:
# importamos librería y df
import pandas as pd
df_fortune = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/fortune1000.csv")
df_fortune.dtypes

Rank          int64
Company      object
Sector       object
Industry     object
Location     object
Revenue       int64
Profits       int64
Employees     int64
dtype: object

### Ejercicio #2


##### Extrear el top 5 planetas (homeworlds) con mayor número de personajes incluídos en el dataset.

> Dataset https://data-wizards.s3.amazonaws.com/datasets/starwarsdb_people.csv


In [ ]:
# importamos librería y df
import pandas as pd
df_starwars_people = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/starwarsdb_people.csv")
df_starwars_people.dtypes

name           object
height        float64
mass          float64
hair_color     object
skin_color     object
eye_color      object
birth_year    float64
gender         object
homeworld      object
species        object
sex            object
dtype: object

### Ejercicio #3

##### Agregar las siguiente métricas según continente,

*   *Población total*
*   *PIB per cápita medio*
*   *media de % población viviendo por debajo de la línea de la pobreza*

> Dataset https://data-wizards.s3.amazonaws.com/datasets/dataset_na_who.csv


In [ ]:
# importamos librería y df
import pandas as pd
df_who = pd.read_csv('https://data-wizards.s3.amazonaws.com/datasets/dataset_na_who.csv')
df_who.dtypes

Country                                                    object
CountryID                                                   int64
ContinentID                                                 int64
Adolescent fertility rate (%)                             float64
Adult literacy rate (%)                                   float64
Gross national income per capita (PPP international $)    float64
Net primary school enrolment ratio female (%)             float64
Net primary school enrolment ratio male (%)               float64
Population (in thousands) total                           float64
Population annual growth rate (%)                         float64
Population in urban areas (%)                             float64
Population living below the poverty line                  float64
Continent                                                  object
dtype: object

### Ejercicio #4

La tabla de datos a importar contiene valores de cotización diarios para el Bitcoin. Agregar DataFrame para calcular media mensual de valor de cotización de apertura.  

> Dataset https://data-wizards.s3.amazonaws.com/datasets/dataset_bitcoin.csv

In [ ]:
# importamos librería y df
import pandas as pd
df_btc = pd.read_csv('https://data-wizards.s3.amazonaws.com/datasets/dataset_bitcoin.csv')
df_btc.dtypes

date           object
Open          float64
High          float64
Low           float64
Close         float64
volume_BTC    float64
volume_usd    float64
dtype: object